In [14]:
%load_ext autoreload
%autoreload 2

from spalah.dataframe import slice_dataframe
from pyspark.sql import SparkSession

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
spark = SparkSession.builder.getOrCreate()

22/08/05 12:32:11 WARN Utils: Your hostname, Alexandrs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.121 instead (on interface en0)
22/08/05 12:32:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/05 12:32:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### SchemaComparer

In [3]:
df_source = spark.sql(
    'SELECT 1 as ID, "John" AS Name, struct("line1" AS Line1, "line2" AS Line2) AS Address'
)
df_source.printSchema()

df_target = spark.sql(
    'SELECT "a" as ID, "John" AS name, struct("line1" AS Line1) AS Address'
)
df_target.printSchema()




root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)
 |    |-- Line2: string (nullable = false)

root
 |-- ID: string (nullable = false)
 |-- name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)



In [12]:
schema_comparer = SchemaComparer(
    source_schema = df_source.schema,
    target_schema = df_target.schema
)

schema_comparer.compare()

schema_comparer.matched
schema_comparer.not_matched

[NotMatchedColumn(name='name', data_type='StringType', reason="The column exists in source and target schemas but it's name is case-mismatched"),
 NotMatchedColumn(name='ID', data_type='IntegerType <=> StringType', reason='The column exists in source and target schemas but it is not matched by a data type'),
 NotMatchedColumn(name='Address.Line2', data_type='StringType', reason='The column exists only in the source schema')]

### flatten_schema

In [9]:
from spalah.dataframe import flatten_schema

In [7]:
df_complex_schema = spark.sql(
    'SELECT 1 as ID, "John" AS Name, struct("line1" AS Line1, "line2" AS Line2) AS Address'
)
df_source.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)
 |    |-- Line2: string (nullable = false)



In [11]:
flatten_schema(
    schema=df_complex_schema.schema,
    include_datatype=True
)

[('ID', 'IntegerType'),
 ('Name', 'StringType'),
 ('Address.Line1', 'StringType'),
 ('Address.Line2', 'StringType')]

### script_dataframe

In [5]:
df = spark.sql(
    'SELECT 1 as ID, "John" AS Name, struct("line1" AS Line1, "line2" AS Line2) AS Address'
)
df.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)
 |    |-- Line2: string (nullable = false)



In [12]:
from spalah.dataframe import script_dataframe

script = script_dataframe(df)

print(script)

from pyspark.sql import Row
import datetime
from decimal import Decimal
from pyspark.sql.types import *

# Scripted data and schema:
__data = [Row(ID=1, Name='John', Address=Row(Line1='line1', Line2='line2'))]

__schema = {'type': 'struct', 'fields': [{'name': 'ID', 'type': 'integer', 'nullable': False, 'metadata': {}}, {'name': 'Name', 'type': 'string', 'nullable': False, 'metadata': {}}, {'name': 'Address', 'type': {'type': 'struct', 'fields': [{'name': 'Line1', 'type': 'string', 'nullable': False, 'metadata': {}}, {'name': 'Line2', 'type': 'string', 'nullable': False, 'metadata': {}}]}, 'nullable': False, 'metadata': {}}]}

outcome_dataframe = spark.createDataFrame(__data, StructType.fromJson(__schema))


### slice_dataframe

In [16]:
from spalah.dataframe import slice_dataframe

df = spark.sql(
    'SELECT 1 as ID, "John" AS Name, struct("line1" AS Line1, "line2" AS Line2) AS Address'
)
df.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)
 |    |-- Line2: string (nullable = false)



In [18]:
df_result = slice_dataframe(
    input_dataframe=df,
    columns_to_include=["Name", "Address"],
    columns_to_exclude=["Address.Line2"]
)

df_result.printSchema()

root
 |-- Name: string (nullable = false)
 |-- Address: struct (nullable = false)
 |    |-- Line1: string (nullable = false)



In [20]:
# alternatively, excluded columns can be nullified instead of removed
df_result = slice_dataframe(
    input_dataframe=df,
    columns_to_include=["Name", "Address"],
    columns_to_exclude=["Address.Line2"],
    nullify_only=True
)

df_result.show()

+----+----+-------------+
|  ID|Name|      Address|
+----+----+-------------+
|null|John|{line1, null}|
+----+----+-------------+

